In [14]:
import pandas as pd
from dateutil import parser

# Import raw data from csv to DataFrame
df = pd.read_csv('data/source/collated_bike_data.csv')
original_len = len(df)
print(f"\n Number of records in source: {original_len}")

# Clean NA vals to appropriate data type
df['name'] = df['name'].fillna('Unknown')
df['email'] = df['email'].fillna('Unknown')
df['birthyear'] = df['birthyear'].fillna(0)

# Remove unnecessary duplicate columns, rename mis-named or mis-cased columns
df = df.drop(['start_time_formatted', 'end_time_formatted'], axis=1)
df.rename(columns={'to_station_id.1': 'to_station_name', 'birthyear': 'birth_year', 'usertype': 'user_type', 'tripduration': 'trip_duration'}, inplace=True)

# Take length of original data set, and print first 50 records
original_len = len(df)
df.head(10)


 Number of records in source: 40000


,trip_id,start_time,end_time,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,birth_year,name,email
0,22014544,2022-03-08 04:19:43,2022-03-08 04:26:42,419,456.0,2112 W Peterson Ave,458.0,Broadway & Thorndale Ave,Subscriber,1992,Frances Clayton,griddled@curtis.com
1,22112934,2022-03-22 15:37:12,2022-03-22 15:41:36,264,456.0,2112 W Peterson Ave,457.0,Clark St & Elmdale Ave,Subscriber,1998,Beth Harris,beth.harris@hotmail.com
2,21757938,2022-01-04 04:18:55,2022-01-04 04:25:27,392,456.0,2112 W Peterson Ave,458.0,Broadway & Thorndale Ave,Subscriber,2001,Andrew Petersen,andrew_petersen@yahoo.com
3,22119398,2022-03-23 11:12:23,2022-03-23 11:35:50,"1,407.00",456.0,2112 W Peterson Ave,447.0,Glenwood Ave & Morse Ave,Subscriber,2003,Tracy Morgan,lealness@yoder.com
4,23503206,2022-07-02 00:45:25,2022-07-02 00:56:50,685,456.0,2112 W Peterson Ave,344.0,Ravenswood Ave & Lawrence Ave,Subscriber,1983,Courtney Howell,courtney_howell@yahoo.com
5,25297896,2022-10-06 03:52:31,2022-10-06 04:09:39,"1,027.00",456.0,2112 W Peterson Ave,463.0,Clark St & Berwyn Ave,Subscriber,1991,Chelsea Horton,chorton@hotmail.com
6,25953758,2022-12-29 04:45:24,2022-12-29 04:49:14,229,456.0,2112 W Peterson Ave,455.0,Maplewood Ave & Peterson Ave,Subscriber,1998,Tricia Hayes,tricia_hayes@flores-mooney.com
7,065A2049EE4C26C3,2/3/2023 12:24,2/3/2023 12:36,NaN,456.0,2112 W Peterson Ave,467.0,Western Ave & Lunt Ave,member,4/13/2006,Carlos Adkins,carlosadkins@gmail.com
8,067CE96BAC83029A,5/19/2023 12:02,5/19/2023 12:20,NaN,456.0,2112 W Peterson Ave,475.0,Washtenaw Ave & Lawrence Ave,member,10/9/1999,Mary Moore,mary.moore@carr.com
9,68C2D846EEC8AEA1,8/5/2023 07:50,8/5/2023 08:08,NaN,456.0,2112 W Peterson Ave,520.0,Greenview Ave & Jarvis Ave,member,6/14/1996,Daniel Hartman,daniel_hartman@yahoo.com


In [15]:
# Fill empty from/to station cells with null val of appropriate data type, set ids to int

df[['from_station_id', 'from_station_name', 'to_station_id', 'to_station_name']] = (
    df[['from_station_id', 'from_station_name', 'to_station_id', 'to_station_name']]
    .fillna({'from_station_id': 0, 'from_station_name': 'Unknown', 'to_station_id': 0, 'to_station_name': 'Unknown'})
)
df[['from_station_id', 'to_station_id']] = df[['from_station_id', 'to_station_id']].astype(int)

#Check data
df.head(10)

,trip_id,start_time,end_time,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,birth_year,name,email
0,22014544,2022-03-08 04:19:43,2022-03-08 04:26:42,419,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,1992,Frances Clayton,griddled@curtis.com
1,22112934,2022-03-22 15:37:12,2022-03-22 15:41:36,264,456,2112 W Peterson Ave,457,Clark St & Elmdale Ave,Subscriber,1998,Beth Harris,beth.harris@hotmail.com
2,21757938,2022-01-04 04:18:55,2022-01-04 04:25:27,392,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,2001,Andrew Petersen,andrew_petersen@yahoo.com
3,22119398,2022-03-23 11:12:23,2022-03-23 11:35:50,"1,407.00",456,2112 W Peterson Ave,447,Glenwood Ave & Morse Ave,Subscriber,2003,Tracy Morgan,lealness@yoder.com
4,23503206,2022-07-02 00:45:25,2022-07-02 00:56:50,685,456,2112 W Peterson Ave,344,Ravenswood Ave & Lawrence Ave,Subscriber,1983,Courtney Howell,courtney_howell@yahoo.com
5,25297896,2022-10-06 03:52:31,2022-10-06 04:09:39,"1,027.00",456,2112 W Peterson Ave,463,Clark St & Berwyn Ave,Subscriber,1991,Chelsea Horton,chorton@hotmail.com
6,25953758,2022-12-29 04:45:24,2022-12-29 04:49:14,229,456,2112 W Peterson Ave,455,Maplewood Ave & Peterson Ave,Subscriber,1998,Tricia Hayes,tricia_hayes@flores-mooney.com
7,065A2049EE4C26C3,2/3/2023 12:24,2/3/2023 12:36,NaN,456,2112 W Peterson Ave,467,Western Ave & Lunt Ave,member,4/13/2006,Carlos Adkins,carlosadkins@gmail.com
8,067CE96BAC83029A,5/19/2023 12:02,5/19/2023 12:20,NaN,456,2112 W Peterson Ave,475,Washtenaw Ave & Lawrence Ave,member,10/9/1999,Mary Moore,mary.moore@carr.com
9,68C2D846EEC8AEA1,8/5/2023 07:50,8/5/2023 08:08,NaN,456,2112 W Peterson Ave,520,Greenview Ave & Jarvis Ave,member,6/14/1996,Daniel Hartman,daniel_hartman@yahoo.com


In [16]:
# Convert date-like strings to datetime values

df['start_time'] = pd.to_datetime(df['start_time'], format='mixed')
df['end_time'] = pd.to_datetime(df['end_time'], format='mixed')

start_time = df['start_time'].min()
end_time = df['end_time'].max()
print(f"The first bike ride started at: {start_time}, and the last ended at {end_time}")

# Check data
df.head(10)

The first bike ride started at: 2021-12-31 19:13:32, and the last ended at 2023-12-01 13:51:00


,trip_id,start_time,end_time,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,birth_year,name,email
0,22014544,2022-03-08 04:19:43,2022-03-08 04:26:42,419,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,1992,Frances Clayton,griddled@curtis.com
1,22112934,2022-03-22 15:37:12,2022-03-22 15:41:36,264,456,2112 W Peterson Ave,457,Clark St & Elmdale Ave,Subscriber,1998,Beth Harris,beth.harris@hotmail.com
2,21757938,2022-01-04 04:18:55,2022-01-04 04:25:27,392,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,2001,Andrew Petersen,andrew_petersen@yahoo.com
3,22119398,2022-03-23 11:12:23,2022-03-23 11:35:50,"1,407.00",456,2112 W Peterson Ave,447,Glenwood Ave & Morse Ave,Subscriber,2003,Tracy Morgan,lealness@yoder.com
4,23503206,2022-07-02 00:45:25,2022-07-02 00:56:50,685,456,2112 W Peterson Ave,344,Ravenswood Ave & Lawrence Ave,Subscriber,1983,Courtney Howell,courtney_howell@yahoo.com
5,25297896,2022-10-06 03:52:31,2022-10-06 04:09:39,"1,027.00",456,2112 W Peterson Ave,463,Clark St & Berwyn Ave,Subscriber,1991,Chelsea Horton,chorton@hotmail.com
6,25953758,2022-12-29 04:45:24,2022-12-29 04:49:14,229,456,2112 W Peterson Ave,455,Maplewood Ave & Peterson Ave,Subscriber,1998,Tricia Hayes,tricia_hayes@flores-mooney.com
7,065A2049EE4C26C3,2023-02-03 12:24:00,2023-02-03 12:36:00,NaN,456,2112 W Peterson Ave,467,Western Ave & Lunt Ave,member,4/13/2006,Carlos Adkins,carlosadkins@gmail.com
8,067CE96BAC83029A,2023-05-19 12:02:00,2023-05-19 12:20:00,NaN,456,2112 W Peterson Ave,475,Washtenaw Ave & Lawrence Ave,member,10/9/1999,Mary Moore,mary.moore@carr.com
9,68C2D846EEC8AEA1,2023-08-05 07:50:00,2023-08-05 08:08:00,NaN,456,2112 W Peterson Ave,520,Greenview Ave & Jarvis Ave,member,6/14/1996,Daniel Hartman,daniel_hartman@yahoo.com


In [17]:
# Calculate tripduration in minutes from end_time/start_time, return as integer
df['trip_duration'] = (df['end_time'] - df['start_time']) / pd.Timedelta(minutes=1)
df['trip_duration'] = df['trip_duration'].astype(int)
df.head(10)

,trip_id,start_time,end_time,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,birth_year,name,email
0,22014544,2022-03-08 04:19:43,2022-03-08 04:26:42,6,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,1992,Frances Clayton,griddled@curtis.com
1,22112934,2022-03-22 15:37:12,2022-03-22 15:41:36,4,456,2112 W Peterson Ave,457,Clark St & Elmdale Ave,Subscriber,1998,Beth Harris,beth.harris@hotmail.com
2,21757938,2022-01-04 04:18:55,2022-01-04 04:25:27,6,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,2001,Andrew Petersen,andrew_petersen@yahoo.com
3,22119398,2022-03-23 11:12:23,2022-03-23 11:35:50,23,456,2112 W Peterson Ave,447,Glenwood Ave & Morse Ave,Subscriber,2003,Tracy Morgan,lealness@yoder.com
4,23503206,2022-07-02 00:45:25,2022-07-02 00:56:50,11,456,2112 W Peterson Ave,344,Ravenswood Ave & Lawrence Ave,Subscriber,1983,Courtney Howell,courtney_howell@yahoo.com
5,25297896,2022-10-06 03:52:31,2022-10-06 04:09:39,17,456,2112 W Peterson Ave,463,Clark St & Berwyn Ave,Subscriber,1991,Chelsea Horton,chorton@hotmail.com
6,25953758,2022-12-29 04:45:24,2022-12-29 04:49:14,3,456,2112 W Peterson Ave,455,Maplewood Ave & Peterson Ave,Subscriber,1998,Tricia Hayes,tricia_hayes@flores-mooney.com
7,065A2049EE4C26C3,2023-02-03 12:24:00,2023-02-03 12:36:00,12,456,2112 W Peterson Ave,467,Western Ave & Lunt Ave,member,4/13/2006,Carlos Adkins,carlosadkins@gmail.com
8,067CE96BAC83029A,2023-05-19 12:02:00,2023-05-19 12:20:00,18,456,2112 W Peterson Ave,475,Washtenaw Ave & Lawrence Ave,member,10/9/1999,Mary Moore,mary.moore@carr.com
9,68C2D846EEC8AEA1,2023-08-05 07:50:00,2023-08-05 08:08:00,18,456,2112 W Peterson Ave,520,Greenview Ave & Jarvis Ave,member,6/14/1996,Daniel Hartman,daniel_hartman@yahoo.com


In [18]:
# Where birthyear is a string val, parse to date, and take the year, else return the original val
df['birth_year'] = (
    df['birth_year']
    .apply(lambda x: int(parser.parse(x, fuzzy=True).year) if isinstance(x, str) else x)
)
df.head(10)

,trip_id,start_time,end_time,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,birth_year,name,email
0,22014544,2022-03-08 04:19:43,2022-03-08 04:26:42,6,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,1992,Frances Clayton,griddled@curtis.com
1,22112934,2022-03-22 15:37:12,2022-03-22 15:41:36,4,456,2112 W Peterson Ave,457,Clark St & Elmdale Ave,Subscriber,1998,Beth Harris,beth.harris@hotmail.com
2,21757938,2022-01-04 04:18:55,2022-01-04 04:25:27,6,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,2001,Andrew Petersen,andrew_petersen@yahoo.com
3,22119398,2022-03-23 11:12:23,2022-03-23 11:35:50,23,456,2112 W Peterson Ave,447,Glenwood Ave & Morse Ave,Subscriber,2003,Tracy Morgan,lealness@yoder.com
4,23503206,2022-07-02 00:45:25,2022-07-02 00:56:50,11,456,2112 W Peterson Ave,344,Ravenswood Ave & Lawrence Ave,Subscriber,1983,Courtney Howell,courtney_howell@yahoo.com
5,25297896,2022-10-06 03:52:31,2022-10-06 04:09:39,17,456,2112 W Peterson Ave,463,Clark St & Berwyn Ave,Subscriber,1991,Chelsea Horton,chorton@hotmail.com
6,25953758,2022-12-29 04:45:24,2022-12-29 04:49:14,3,456,2112 W Peterson Ave,455,Maplewood Ave & Peterson Ave,Subscriber,1998,Tricia Hayes,tricia_hayes@flores-mooney.com
7,065A2049EE4C26C3,2023-02-03 12:24:00,2023-02-03 12:36:00,12,456,2112 W Peterson Ave,467,Western Ave & Lunt Ave,member,2006,Carlos Adkins,carlosadkins@gmail.com
8,067CE96BAC83029A,2023-05-19 12:02:00,2023-05-19 12:20:00,18,456,2112 W Peterson Ave,475,Washtenaw Ave & Lawrence Ave,member,1999,Mary Moore,mary.moore@carr.com
9,68C2D846EEC8AEA1,2023-08-05 07:50:00,2023-08-05 08:08:00,18,456,2112 W Peterson Ave,520,Greenview Ave & Jarvis Ave,member,1996,Daniel Hartman,daniel_hartman@yahoo.com


In [19]:
# Helper func to set every member/casual as a Subscriber/Customer - will also capitalise Subscriber/Customers with wrong casing

def clean_user_type(user):
    return (
        user.lower()
        .replace('member', 'Subscriber')
        .replace('casual', 'Customer')
        .capitalize()
    )
df['user_type'] = df['user_type'].apply(clean_user_type)

df.head(10)

,trip_id,start_time,end_time,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,birth_year,name,email
0,22014544,2022-03-08 04:19:43,2022-03-08 04:26:42,6,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,1992,Frances Clayton,griddled@curtis.com
1,22112934,2022-03-22 15:37:12,2022-03-22 15:41:36,4,456,2112 W Peterson Ave,457,Clark St & Elmdale Ave,Subscriber,1998,Beth Harris,beth.harris@hotmail.com
2,21757938,2022-01-04 04:18:55,2022-01-04 04:25:27,6,456,2112 W Peterson Ave,458,Broadway & Thorndale Ave,Subscriber,2001,Andrew Petersen,andrew_petersen@yahoo.com
3,22119398,2022-03-23 11:12:23,2022-03-23 11:35:50,23,456,2112 W Peterson Ave,447,Glenwood Ave & Morse Ave,Subscriber,2003,Tracy Morgan,lealness@yoder.com
4,23503206,2022-07-02 00:45:25,2022-07-02 00:56:50,11,456,2112 W Peterson Ave,344,Ravenswood Ave & Lawrence Ave,Subscriber,1983,Courtney Howell,courtney_howell@yahoo.com
5,25297896,2022-10-06 03:52:31,2022-10-06 04:09:39,17,456,2112 W Peterson Ave,463,Clark St & Berwyn Ave,Subscriber,1991,Chelsea Horton,chorton@hotmail.com
6,25953758,2022-12-29 04:45:24,2022-12-29 04:49:14,3,456,2112 W Peterson Ave,455,Maplewood Ave & Peterson Ave,Subscriber,1998,Tricia Hayes,tricia_hayes@flores-mooney.com
7,065A2049EE4C26C3,2023-02-03 12:24:00,2023-02-03 12:36:00,12,456,2112 W Peterson Ave,467,Western Ave & Lunt Ave,Subscriber,2006,Carlos Adkins,carlosadkins@gmail.com
8,067CE96BAC83029A,2023-05-19 12:02:00,2023-05-19 12:20:00,18,456,2112 W Peterson Ave,475,Washtenaw Ave & Lawrence Ave,Subscriber,1999,Mary Moore,mary.moore@carr.com
9,68C2D846EEC8AEA1,2023-08-05 07:50:00,2023-08-05 08:08:00,18,456,2112 W Peterson Ave,520,Greenview Ave & Jarvis Ave,Subscriber,1996,Daniel Hartman,daniel_hartman@yahoo.com


In [20]:
# Sanity check number of records is same, print, and check types

number_removed = original_len - len(df)
print(f"\n Number of records in target: {len(df)}")
print(f"\n {number_removed} lines have been removed\n")
print(df.dtypes)

# Export to CSV
df.to_csv('data/target/cleaned_bike_data.csv', index=False)


 Number of records in target: 40000

 0 lines have been removed

trip_id                      object
start_time           datetime64[ns]
end_time             datetime64[ns]
trip_duration                 int64
from_station_id               int64
from_station_name            object
to_station_id                 int64
to_station_name              object
user_type                    object
birth_year                    int64
name                         object
email                        object
dtype: object
